In [16]:
import os
from glob import glob

import pandas

# Fathom flooding
- parse list of files
- output merge commands
- metadata from merged files

In [3]:
df = pandas.read_csv('fathom_data/details.csv', names=['path'])
df

,path
0,by_country/democratic-republic-of-congo/pluvia...
1,by_country/democratic-republic-of-congo/pluvia...
2,by_country/democratic-republic-of-congo/pluvia...
3,by_country/democratic-republic-of-congo/pluvia...
4,by_country/democratic-republic-of-congo/pluvia...
...,...
445,by_country/burundi/fluvial_undefended/FU_1in50...
446,by_country/burundi/fluvial_undefended/FU_1in10...
447,by_country/burundi/fluvial_undefended/FU_1in75...
448,by_country/burundi/fluvial_undefended/FU_1in25...


In [12]:
def parse_type(s):
    path = s.path
    fname = os.path.basename(path)
    hazard, rp = fname.replace(".tif", "").split("_1in")
    return hazard, int(rp)
df[['hazard', 'rp']] = df.apply(parse_type, axis=1, result_type='expand')

In [10]:
def parse_country(path):
    country = os.path.basename(os.path.dirname(os.path.dirname(path)))
    return country.replace("-", " ").title().replace("Of", "of")
df['country'] = df.path.apply(parse_country)

In [13]:
rps = sorted(df.rp.unique())
hazards = sorted(df.hazard.unique())
rps, hazards

([5, 10, 20, 50, 75, 100, 200, 250, 500, 1000], ['FD', 'FU', 'P'])

In [15]:
with open('fathom_data/merge.sh', 'w') as fh:
    for rp in rps:
        for hazard in hazards:
            if hazard == 'FD':
                continue
            paths = list(df[(df.rp == rp) & (df.hazard == hazard)].path)
            cmd = f"""gdal_merge.py -co "COMPRESS=LZW" -o {hazard}__rp_{rp}.tif {' '.join(paths)}\n"""
            fh.write(cmd)

In [18]:
merged_fnames = sorted(glob('fathom_data/*.tif'))

In [34]:
floods = pandas.DataFrame({'path': merged_fnames})
def parse_merged(s):
    path = s.path
    # FU__rp_10
    fname = os.path.basename(path)
    hazard, rp = fname.replace(".tif", "").split("__rp_")
    return hazard, int(rp)
floods[['hazard', 'rp']] = floods.apply(parse_merged, axis=1, result_type='expand')
floods['ssp'] = 'historical'
floods.to_csv('details_fathom.csv', index=False)
floods

,path,hazard,rp,ssp
0,fathom_data/FU__rp_10.tif,FU,10,historical
1,fathom_data/FU__rp_100.tif,FU,100,historical
2,fathom_data/FU__rp_1000.tif,FU,1000,historical
3,fathom_data/FU__rp_20.tif,FU,20,historical
4,fathom_data/FU__rp_200.tif,FU,200,historical
5,fathom_data/FU__rp_250.tif,FU,250,historical
6,fathom_data/FU__rp_5.tif,FU,5,historical
7,fathom_data/FU__rp_50.tif,FU,50,historical
8,fathom_data/FU__rp_500.tif,FU,500,historical
9,fathom_data/FU__rp_75.tif,FU,75,historical


# Extreme heat
- parse meta from nc files

In [25]:
heat_fnames = sorted(glob('world*/*.nc'))

In [35]:
heat = pandas.DataFrame({'path':heat_fnames})
heat[['hazard','ssp','quantile','epoch']] = \
    heat.path.str.extract(
        r'climatology-(.*)-annual-mean_cmip6_annual_all-regridded-bct-(.*)-climatology_(.*)_(.*).nc', 
        expand=True)

heat.to_csv('details_heat.csv', index=False)
heat

,path,hazard,ssp,quantile,epoch
0,world-bank-climate-knowledge-portal/climatolog...,hd35,historical,median,1995-2014
1,world-bank-climate-knowledge-portal/climatolog...,hd35,historical,p10,1995-2014
2,world-bank-climate-knowledge-portal/climatolog...,hd35,historical,p90,1995-2014
3,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp126,median,2040-2059
4,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp126,p10,2040-2059
5,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp126,p90,2040-2059
6,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp245,median,2040-2059
7,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp245,p10,2040-2059
8,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp245,p90,2040-2059
9,world-bank-climate-knowledge-portal/climatolog...,hd35,ssp585,median,2040-2059
